In [1]:
from multiprocess import pool
import lib.aoc.grid2d.grid as grid2d
import lib.aoc.grid2d.vector as vector2d

In [2]:
with open('../data/2024/day6.txt') as f:
   data = f.read()

In [3]:
grid = grid2d.parse(data)

In [4]:
obstacles = grid2d.find_coords(grid, '#')

In [5]:
# Part 1
visited = set()
guard_at = next(iter(grid2d.find_coords(grid, '^')))
guard_dir = (-1,0) # north

while True:
    # Keep track of our past steps (plus starting location)
    visited.add(guard_at)

    # Look ahead
    next_step = vector2d.add(guard_at, guard_dir)

    # If the next step is out of bounds, we're done
    if not grid.get(next_step): break
    # If we hit an obstacle, rotate clockwise 90º
    elif next_step in obstacles: guard_dir = vector2d.rotate(guard_dir, 'R')
    # Otherwise move
    else: guard_at = next_step

print("Part 1: {}".format(len(visited)))

Part 1: 4656


In [6]:
# Part 2
# Only try positions we previously walked in part 1
guard_start = next(iter(grid2d.find_coords(grid, '^')))
prospective_locations = set(visited).difference([guard_start])

def guard_walk(new_obstacle):
    # Reset our state every attempt
    visited = set()
    guard_at = guard_start
    guard_dir = (-1,0) # north

    # Clone the original obstacles and add our new one
    prospective_obstacles = set(obstacles)
    prospective_obstacles.add(new_obstacle)

    while True:
        # If we repeat a location + heading we found a cycle
        if (*guard_at, guard_dir) in visited:
            return new_obstacle

        # Keep track of our past steps and headings
        visited.add((*guard_at, guard_dir))

        # Look ahead
        next_step = vector2d.add(guard_at, guard_dir)

        # If the next step is out of bounds, abort this attempt
        if not grid.get(next_step): break

        # If we hit an obstacle, rotate clockwise 90º
        elif next_step in prospective_obstacles: guard_dir = vector2d.rotate(guard_dir, 'R')
        # Otherwise move
        else: guard_at = next_step

with pool.Pool() as p:
    results = p.map(guard_walk, prospective_locations)

print("Part 2:", len(list(filter(None, results))))

Part 2: 1575
